<h1>Overview<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spooky-Author-Classification" data-toc-modified-id="Spooky-Author-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Spooky Author Classification</a></span><ul class="toc-item"><li><span><a href="#Generate-Tokenizer-Input-Data" data-toc-modified-id="Generate-Tokenizer-Input-Data-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generate Tokenizer Input Data</a></span></li><li><span><a href="#Generate-Tokenizer" data-toc-modified-id="Generate-Tokenizer-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Generate Tokenizer</a></span></li><li><span><a href="#Tokenize-texts" data-toc-modified-id="Tokenize-texts-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Tokenize texts</a></span></li><li><span><a href="#Add-a-Regular-Tokenizer-to-compare" data-toc-modified-id="Add-a-Regular-Tokenizer-to-compare-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Add a Regular Tokenizer to compare</a></span></li></ul></li><li><span><a href="#Estimate-a-reasonable-sequence's-max_length" data-toc-modified-id="Estimate-a-reasonable-sequence's-max_length-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Estimate a reasonable sequence's max_length</a></span><ul class="toc-item"><li><span><a href="#Enforce-a-sequence's-max_length" data-toc-modified-id="Enforce-a-sequence's-max_length-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Enforce a sequence's max_length</a></span></li><li><span><a href="#Create-train-/test-sets" data-toc-modified-id="Create-train-/test-sets-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Create train-/test sets</a></span></li><li><span><a href="#Create-Model" data-toc-modified-id="Create-Model-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Create Model</a></span></li><li><span><a href="#Fit-Model" data-toc-modified-id="Fit-Model-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Fit Model</a></span></li><li><span><a href="#Evaluate-Model" data-toc-modified-id="Evaluate-Model-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Evaluate Model</a></span></li></ul></li></ul></div>

In [1]:
import os
import re
import sys
import numpy as np
import pandas as pd

from keras import backend as K
from keras.layers import Dense, Conv1D, LSTM, Dropout, Embedding, Layer
from keras.models import Sequential as SequentialModel
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer as KerasTokenizer

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '../ct')

import load
from preprocess import preprocess
from preprocess import Tokenizer
from preprocess.preprocess import separator_samples

Using TensorFlow backend.


In [2]:
# display(f'current dir: {os.getcwd()}')
# df = pd.read_pickle('../data/processed/treebank.pickle')

# if not os.path.exists('../data/processed/treebank'):
#     os.mkdir('../data/processed/treebank')

# for i, text in enumerate(df.text):
#     with open(f'../data/processed/treebank/{i + 1}', 'w') as file:
#         file.write(text)
        
# with open('../data/processed/treebank.txt', 'w', ) as file:
#     file.write(text)

# tokenizer = Tokenizer(input_paths=['../data/input/treebank/raw_txt/treebank-utf8.txt'],
#                       tokens_output_dir='../data/processed/treebank/',
#                       tokenizer_output_path='../data/tokenizer/treebank',
#                       lowercase=True,
#                       vocab_size=5000)
# display(tokenizer)

# Spooky Author Classification

## Generate Tokenizer Input Data

In [37]:
spooky_train = pd.read_csv('../data/input/spooky-author/train.csv')
spooky_test = pd.read_csv('../data/input/spooky-author/test.csv')

In [38]:
spooky_train

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL
...,...,...,...
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [39]:
if not os.path.exists('../data/processed/spooky-author'):
    os.mkdir('../data/processed/spooky-author')

x_train = f'{separator_samples}'.join(spooky_train.text)
x_train = x_train.encode('utf-8', 'backslashreplace').decode('utf-8', 'backslashreplace')

with open('../data/input/spooky-author/train.txt', 'w', encoding="utf-8") as file:
    file.write(x_train)

## Generate Tokenizer

In [40]:
tokenizer = Tokenizer(input_paths=['../data/input/spooky-author/train.txt'],
                      tokenizer_output_path='../data/tokenizer/spooky-author',
                      lowercase=True,
                      vocab_size=4000)

## Tokenize texts

In [41]:
spooky_train['encoding'] = tokenizer.encode_batch(spooky_train.text.tolist())
spooky_test['encoding'] = tokenizer.encode_batch(spooky_test.text.tolist())

spooky_train['tokens'] = spooky_train.encoding.apply(lambda e: e.tokens)
spooky_train['token_ids'] = spooky_train.encoding.apply(lambda e: e.ids)

spooky_test['tokens'] = spooky_test.encoding.apply(lambda e: e.tokens)
spooky_test['token_ids'] = spooky_test.encoding.apply(lambda e: e.ids)

author_to_id = {'EAP': 0, 'HPL': 1, 'MWS': 2}
spooky_train['author_id'] = spooky_train.author.apply(lambda a: author_to_id[a])

In [8]:
for i, row in spooky_train.head(20).iterrows():
    print(row.text)
    display(row.tokens)
    print()

This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.


['th',
 'is',
 'Ġpro',
 'c',
 'ess',
 ',',
 'Ġh',
 'ow',
 'e',
 'ver',
 ',',
 'Ġa',
 'ff',
 'or',
 'd',
 'ed',
 'Ġme',
 'Ġno',
 'Ġme',
 'an',
 's',
 'Ġof',
 'Ġas',
 'c',
 'er',
 't',
 'ain',
 'ing',
 'Ġthe',
 'Ġd',
 'im',
 'en',
 's',
 'ion',
 's',
 'Ġof',
 'Ġmy',
 'Ġd',
 'u',
 'n',
 'ge',
 'on',
 ';',
 'Ġas',
 'Ġi',
 'Ġm',
 'ight',
 'Ġm',
 'a',
 'ke',
 'Ġit',
 's',
 'Ġc',
 'ir',
 'c',
 'u',
 'it',
 ',',
 'Ġand',
 'Ġre',
 't',
 'ur',
 'n',
 'Ġto',
 'Ġthe',
 'Ġpo',
 'in',
 't',
 'Ġwhen',
 'ce',
 'Ġi',
 'Ġs',
 'et',
 'Ġo',
 'ut',
 ',',
 'Ġwith',
 'out',
 'Ġbe',
 'ing',
 'Ġa',
 'w',
 'a',
 're',
 'Ġof',
 'Ġthe',
 'Ġf',
 'a',
 'ct',
 ';',
 'Ġso',
 'Ġper',
 'f',
 'ect',
 'ly',
 'Ġun',
 'if',
 'or',
 'm',
 'Ġse',
 'em',
 'ed',
 'Ġthe',
 'Ġw',
 'all',
 '.']


It never once occurred to me that the fumbling might be a mere mistake.


['it',
 'Ġne',
 'ver',
 'Ġon',
 'ce',
 'Ġo',
 'c',
 'c',
 'ur',
 'red',
 'Ġto',
 'Ġme',
 'Ġthat',
 'Ġthe',
 'Ġf',
 'um',
 'b',
 'l',
 'ing',
 'Ġm',
 'ight',
 'Ġbe',
 'Ġa',
 'Ġme',
 're',
 'Ġm',
 'ist',
 'a',
 'ke',
 '.']


In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.


...

['her',
 'Ġn',
 'at',
 'ive',
 'Ġsp',
 'r',
 'ight',
 'l',
 'in',
 'ess',
 'Ġne',
 'ed',
 'ed',
 'Ġno',
 'Ġu',
 'nd',
 'u',
 'e',
 'Ġex',
 'c',
 'it',
 'em',
 'ent',
 ',',
 'Ġand',
 'Ġher',
 'Ġpl',
 'ac',
 'id',
 'Ġhe',
 'art',
 'Ġre',
 'p',
 'o',
 's',
 'ed',
 'Ġcon',
 't',
 'ent',
 'ed',
 'Ġon',
 'Ġmy',
 'Ġlo',
 've',
 ',',
 'Ġthe',
 'Ġwe',
 'll',
 'Ġbe',
 'ing',
 'Ġof',
 'Ġher',
 'Ġc',
 'h',
 'i',
 'ld',
 're',
 'n',
 ',',
 'Ġand',
 'Ġthe',
 'Ġbe',
 'a',
 'ut',
 'y',
 'Ġof',
 'Ġs',
 'ur',
 'r',
 'ound',
 'ing',
 'Ġn',
 'at',
 'ure',
 '.']


I even went so far as to speak of a slightly hectic cough with which, at one time, I had been troubled of a chronic rheumatism of a twinge of hereditary gout and, in conclusion, of the disagreeable and inconvenient, but hitherto carefully concealed, weakness of my eyes.


['i',
 'Ġe',
 'ven',
 'Ġw',
 'ent',
 'Ġso',
 'Ġf',
 'ar',
 'Ġas',
 'Ġto',
 'Ġs',
 'pe',
 'a',
 'k',
 'Ġof',
 'Ġa',
 'Ġs',
 'l',
 'ight',
 'ly',
 'Ġhe',
 'ct',
 'ic',
 'Ġc',
 'ough',
 'Ġwith',
 'Ġwhich',
 ',',
 'Ġat',
 'Ġone',
 'Ġt',
 'im',
 'e',
 ',',
 'Ġi',
 'Ġhad',
 'Ġbeen',
 'Ġt',
 'r',
 'ou',
 'ble',
 'd',
 'Ġof',
 'Ġa',
 'Ġc',
 'h',
 'r',
 'on',
 'ic',
 'Ġr',
 'he',
 'um',
 'at',
 'is',
 'm',
 'Ġof',
 'Ġa',
 'Ġt',
 'w',
 'ing',
 'e',
 'Ġof',
 'Ġhe',
 'red',
 'it',
 'ar',
 'y',
 'Ġg',
 'out',
 'Ġand',
 ',',
 'Ġin',
 'Ġcon',
 'c',
 'l',
 'us',
 'ion',
 ',',
 'Ġof',
 'Ġthe',
 'Ġdis',
 'ag',
 're',
 'e',
 'able',
 'Ġand',
 'Ġin',
 'c',
 'on',
 'ven',
 'i',
 'ent',
 ',',
 'Ġbut',
 'Ġh',
 'it',
 'her',
 't',
 'o',
 'Ġc',
 'a',
 're',
 'f',
 'u',
 'll',
 'y',
 'Ġcon',
 'ce',
 'a',
 'led',
 ',',
 'Ġwe',
 'a',
 'k',
 'ne',
 'ss',
 'Ġof',
 'Ġmy',
 'Ġe',
 'y',
 'es',
 '.']


His facial aspect, too, was remarkable for its maturity; for though he shared his mother's and grandfather's chinlessness, his firm and precociously shaped nose united with the expression of his large, dark, almost Latin eyes to give him an air of quasi adulthood and well nigh preternatural intelligence.


['h',
 'is',
 'Ġf',
 'ac',
 'i',
 'al',
 'Ġas',
 'pe',
 'ct',
 ',',
 'Ġto',
 'o',
 ',',
 'Ġwas',
 'Ġre',
 'm',
 'ar',
 'k',
 'able',
 'Ġfor',
 'Ġit',
 's',
 'Ġm',
 'at',
 'ur',
 'ity',
 ';',
 'Ġfor',
 'Ġth',
 'ough',
 'Ġhe',
 'Ġs',
 'ha',
 'red',
 'Ġhis',
 'Ġm',
 'ot',
 'her',
 "'s",
 'Ġand',
 'Ġg',
 'ra',
 'nd',
 'f',
 'at',
 'her',
 "'s",
 'Ġc',
 'h',
 'in',
 'le',
 'ss',
 'ne',
 'ss',
 ',',
 'Ġhis',
 'Ġf',
 'ir',
 'm',
 'Ġand',
 'Ġpre',
 'c',
 'o',
 'c',
 'i',
 'ous',
 'ly',
 'Ġs',
 'ha',
 'p',
 'ed',
 'Ġno',
 'se',
 'Ġun',
 'it',
 'ed',
 'Ġwith',
 'Ġthe',
 'Ġex',
 'p',
 're',
 'ss',
 'ion',
 'Ġof',
 'Ġhis',
 'Ġl',
 'ar',
 'ge',
 ',',
 'Ġd',
 'ar',
 'k',
 ',',
 'Ġal',
 'm',
 'o',
 'st',
 'Ġl',
 'at',
 'in',
 'Ġe',
 'y',
 'es',
 'Ġto',
 'Ġg',
 'ive',
 'Ġhim',
 'Ġan',
 'Ġa',
 'ir',
 'Ġof',
 'Ġ',
 'qu',
 'as',
 'i',
 'Ġa',
 'd',
 'ul',
 'th',
 'o',
 'od',
 'Ġand',
 'Ġwe',
 'll',
 'Ġn',
 'i',
 'gh',
 'Ġpre',
 'ter',
 'n',
 'at',
 'ur',
 'al',
 'Ġint',
 'e',
 'll',
 'ig',
 'en',
 'ce',
 

In [43]:
a = [tokenizer.encode(f'{separator_samples}{t}').tokens for t in spooky_train.text]

In [44]:
a[0]

['############<',
 'new',
 '_',
 'sample',
 '>############',
 'this',
 'Ġpro',
 'cess',
 ',',
 'Ġhowever',
 ',',
 'Ġafforded',
 'Ġme',
 'Ġno',
 'Ġmeans',
 'Ġof',
 'Ġasc',
 'ertain',
 'ing',
 'Ġthe',
 'Ġdimens',
 'ions',
 'Ġof',
 'Ġmy',
 'Ġdun',
 'ge',
 'on',
 ';',
 'Ġas',
 'Ġi',
 'Ġmight',
 'Ġmake',
 'Ġits',
 'Ġcirc',
 'uit',
 ',',
 'Ġand',
 'Ġreturn',
 'Ġto',
 'Ġthe',
 'Ġpoint',
 'Ġwhen',
 'ce',
 'Ġi',
 'Ġset',
 'Ġout',
 ',',
 'Ġwithout',
 'Ġbeing',
 'Ġaware',
 'Ġof',
 'Ġthe',
 'Ġfact',
 ';',
 'Ġso',
 'Ġperfect',
 'ly',
 'Ġun',
 'if',
 'orm',
 'Ġseemed',
 'Ġthe',
 'Ġwall',
 '.']

## Add a Regular Tokenizer to compare

In [11]:
tokenizer = KerasTokenizer(num_words=15000,
                          lower=True,
                          char_level=False)
tokenizer.fit_on_texts(spooky_train.text)

spooky_train['keras_token_ids'] = tokenizer.texts_to_sequences(spooky_train.text)
spooky_train.keras_token_ids = [[min(s, tokenizer.num_words) for s in seq] for seq in spooky_train.keras_token_ids]

spooky_test['keras_token_ids'] = tokenizer.texts_to_sequences(spooky_test.text)
spooky_test.keras_token_ids = [[min(s, tokenizer.num_words) for s in seq] for seq in spooky_test.keras_token_ids]

# Estimate a reasonable sequence's max_length

In [46]:
lengths = pd.Series([len(t) for t in spooky_train.token_ids])

lengths.quantile(np.linspace(0, 1, num=10))

0.000000       5.0
0.111111      14.0
0.222222      20.0
0.333333      25.0
0.444444      31.0
0.555556      37.0
0.666667      43.0
0.777778      53.0
0.888889      68.0
1.000000    1081.0
dtype: float64

## Enforce a sequence's max_length

In [48]:
# `tokenizer` tokenizer
max_length = 50

spooky_train.token_ids = spooky_train.token_ids.apply(lambda a: (a + [0]*(max_length - len(a)))[:max_length])
spooky_test.token_ids = spooky_test.token_ids.apply(lambda a: (a + [0]*(max_length - len(a)))[:max_length])

In [ ]:
# Keras tokenizer
max_length = 50

lengths = pd.Series([len(tokens) for tokens in spooky_train.keras_token_ids])
spooky_train.keras_token_ids = spooky_train.keras_token_ids.apply(lambda a: (a + [0]*(max_length - len(a)))[:max_length])

spooky_test.token_ids = spooky_test.keras_token_ids.apply(lambda a: (a + [0]*(max_length - len(a)))[:max_length])

## Create train-/test sets

In [49]:
token_column = 'token_ids'

assert token_column in ['token_ids', 'keras_token_ids']

In [50]:
x_train = np.array(spooky_train[token_column].tolist())
y_train = to_categorical(np.array(spooky_train.author_id.tolist()))

x_test = np.array(spooky_test[token_column].tolist())

In [51]:
permutation = np.random.permutation(len(x_train))
x_train = x_train[permutation]
y_train = y_train[permutation]

## Create Model

In [63]:
num_words = tokenizer.get_vocab_size() if not isinstance(tokenizer, KerasTokenizer) else tokenizer.num_words

# create model
model = SequentialModel()
model.add(Embedding(input_dim=num_words, output_dim=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.15))
model.add(Dense(100))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

## Fit Model

In [60]:
# fit model
model.fit(x=x_train, 
          y=y_train,
          validation_split=0.3,
          batch_size=32,
          epochs=10)

C:\Users\Sgt-Peppers\Anaconda3\envs\keras\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 13705 samples, validate on 5874 samples
Epoch 1/10
13705/13705 [==============================] - 34s 2ms/step - loss: 0.8769 - accuracy: 0.5731 - val_loss: 0.7038 - val_accuracy: 0.7433
Epoch 2/10
13705/13705 [==============================] - 33s 2ms/step - loss: 0.5751 - accuracy: 0.7780 - val_loss: 0.5932 - val_accuracy: 0.7622
Epoch 3/10
13705/13705 [==============================] - 35s 3ms/step - loss: 0.4477 - accuracy: 0.8336 - val_loss: 0.5662 - val_accuracy: 0.7828
Epoch 4/10
13705/13705 [==============================] - 33s 2ms/step - loss: 0.3798 - accuracy: 0.8698 - val_loss: 0.5904 - val_accuracy: 0.7802
Epoch 5/10
13705/13705 [==============================] - 32s 2ms/step - loss: 0.3238 - accuracy: 0.8900 - val_loss: 0.6532 - val_accuracy: 0.7702
Epoch 6/10
13705/13705 [==============================] - 44s 3ms/step - loss: 0.2872 - accuracy: 0.9036 - val_loss: 0.6719 - val_accuracy: 0.7659
Epoch 7/10
13705/13705 [==============================] - 47s 3ms/ste

## Evaluate Model

In [ ]:
model.evaluate(x_test, y_test, batch_size=32)